In [1]:
from sqlite3 import adapt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
###################################################################################################################

## macro 데이터 불러오기
trn_macro = pd.read_csv('C:\\Users\\zzuge\\trn_macro.csv')
vld_macro = pd.read_csv('C:\\Users\\zzuge\\vld_macro.csv')

## micro 데이터 불러오기
trn_micro = pd.read_csv('C:\\Users\\zzuge\\trn_micro.csv')
vld_micro = pd.read_csv('C:\\Users\\zzuge\\vld_micro.csv')

trn_macro = trn_macro.drop('product_id', axis = 1)
trn_micro = trn_micro.drop('product_id', axis = 1)
vld_macro = vld_macro.drop('product_id', axis = 1)
vld_micro = vld_micro.drop('product_id', axis = 1)


trn_macro_x = trn_macro.drop('is_applied' ,axis = 1)
trn_macro_y = trn_macro.loc[:,'is_applied']
trn_micro_x = trn_micro.drop('is_applied', axis = 1)
trn_micro_y = trn_micro.loc[:,'is_applied']



## test 검증
vld_total = pd.read_csv('C:\\Users\\zzuge\\vld_total.csv')


###################################################################################################################
## 데이터 학습과정

## test데이터를 macro데이터같은 모양새를 띄도록 변경

real = vld_total['is_applied']
vld_df = vld_total.drop(['is_applied', 'product_id'], axis = 1)
vld_df = vld_df.groupby('application_id').agg('max')
ad = vld_df.index
vld_df = vld_df.reset_index().drop('application_id', axis=1)


## LGBM으로 is_applied 예측해서 하나라도 1이 있는 행들만 모음
## vld_df : application_id와 macro_pred가 합쳐진 데이터프레임
###################################################################################################
macro_lgb = XGBClassifier(random_state=121, n_estimators = 10000)## 분석변경구간#################
#################################################################################################
macro_lgb.fit(trn_macro_x, trn_macro_y)
macro_pred = macro_lgb.predict(vld_df)
macro_pred = pd.DataFrame(macro_pred)
ad = pd.DataFrame(ad)

vld_df = pd.concat([vld_df, ad], axis = 1)
vld_df = pd.concat([vld_df, macro_pred], axis = 1)
vld_df.rename(columns = {0:'macro_pred'}, inplace = True)
vld_df = pd.merge(vld_total, vld_df, left_on = 'application_id', right_on = 'application_id', how = 'inner')

# 나중에 0인자리는 두고 1인 자리에 micro의 예측값을 채워야함
vld_df = vld_df.drop(['credit_score_y',
       'application_n_y', 'loyalty_y', 'gender_y', 'log_yearly_income_y',
       'log_career_y', 'log_desired_amount_y', 'log_existing_loan_cnt_y',
       'log_existing_loan_amt_y', 'log_OpenApp_y', 'log_UseDSRCalc_y',
       'nl_loan_limit_y', 'nl_loan_rate_y', 'nl_age_y', 'is_goodbank_y',
       'is_goodprod_y', 'is_monday_y', 'is_weekend_y', 'is_after_16_y',
       'is_earned_income1_y', 'is_earned_income2_y', 'is_job_regular_y',
       'is_job_etc_y', 'is_house_own_y', 'is_house_lent_y',
       'is_purpose_living_y', 'personal_rehabilitation_yn_y',
       'personal_rehabilitation_complete_yn_y'], axis = 1)

# micro 데이터를 학습을 위한 모양으로 변경
vld_micro_1 = vld_df[vld_df['macro_pred'] == 1]  ## macro 데이터에서 1인것들만 골라서 vld_micro라고 하자
ad = vld_micro_1['application_id']
prd = vld_micro_1['product_id']
## application_id와 product_id와 macro_pred를 빼고 분석시행
vld_micro_1 = vld_micro_1.drop(['application_id', 'product_id', 'macro_pred'], axis = 1)
vld_micro_1 = vld_micro_1.drop('is_applied', axis =1 )
vld_micro_1.columns = ['credit_score', 'application_n', 'loyalty', 'gender',
       'log_yearly_income', 'log_career', 'log_desired_amount',
       'log_existing_loan_cnt', 'log_existing_loan_amt', 'log_OpenApp',
       'log_UseDSRCalc', 'nl_loan_limit', 'nl_loan_rate', 'nl_age',
       'is_goodbank', 'is_goodprod', 'is_monday', 'is_weekend', 'is_after_16',
       'is_earned_income1', 'is_earned_income2', 'is_job_regular',
       'is_job_etc', 'is_house_own', 'is_house_lent', 'is_purpose_living',
       'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn']

## LGBM으로 macro에서 걸러낸 micro데이터의 is_applied를 예측
#################################################################################################
lgb_micro = XGBClassifier(random_state = 121, n_estimators = 10000)##분석변경구간##############
################################################################################################
lgb_micro.fit(trn_micro_x,  trn_micro_y)
micro_pred = lgb_micro.predict_proba(vld_micro_1)

# 임계점을 0.3으로 설정
from sklearn.preprocessing import Binarizer
b = Binarizer(threshold=0.3)
micro_4_pred = b.fit_transform(micro_pred[:,1].reshape(-1,1 )) # 결과가 array값으로 반환

# micro_pred와 vld_micro를 합쳐서 micro_df라고 하자
micro_4_pred = pd.DataFrame(micro_4_pred)
micro_4_pred.rename(columns = {0: 'micro_pred'}, inplace  = True)
vld_micro.reset_index(drop = True, inplace = True)
micro_4_pred.reset_index(drop = True, inplace = True)
vld_micro.index = micro_4_pred.index  # concat과정에서 index가 달라서 nan이 발생해서 인덱스를 동일하게 함
micro_df = pd.concat([vld_micro_1, micro_4_pred], axis =1)

## micro  결과까지 vld_micro데이터프레임에 붙여놓은 상태
# 이제 다시 vld_total 과 같은 형태를 만들어야함

ad.index = micro_df.index
prd.index = micro_df.index  # 떼어놓았던 application_id와 product_id를 다시 micro_df에 병합
micro_df = pd.concat([micro_df, ad], axis = 1 )
micro_df = pd.concat([micro_df, prd], axis = 1 )
micro_df


## vld_df에 1이 있는 곳에 micro_df의 micro_pred 의 값을 채우고 0인곳을 그대로 둔다
vld_0 = vld_df[vld_df['macro_pred']!=1] # macro_pred의 값이 0이 아닌 데이터들을 모은 것
vld_0.drop('is_applied', axis =1 , inplace = True)
vld_0.columns = ['application_id', 'product_id', 'credit_score', 'application_n', 'loyalty', 'gender',
       'log_yearly_income', 'log_career', 'log_desired_amount',
       'log_existing_loan_cnt', 'log_existing_loan_amt', 'log_OpenApp',
       'log_UseDSRCalc', 'nl_loan_limit', 'nl_loan_rate', 'nl_age',
       'is_goodbank', 'is_goodprod', 'is_monday', 'is_weekend', 'is_after_16',
       'is_earned_income1', 'is_earned_income2', 'is_job_regular',
       'is_job_etc', 'is_house_own', 'is_house_lent', 'is_purpose_living',
       'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn', 'macro_pred']

vld_0.reset_index(drop = True, inplace = True)
micro_df.columns
micro_df.rename(columns = {'micro_pred': 'macro_pred'}, inplace = True)
micro_df = micro_df[['application_id', 'product_id','credit_score', 'application_n', 'loyalty', 'gender',
       'log_yearly_income', 'log_career', 'log_desired_amount',
       'log_existing_loan_cnt', 'log_existing_loan_amt', 'log_OpenApp',
       'log_UseDSRCalc', 'nl_loan_limit', 'nl_loan_rate', 'nl_age',
       'is_goodbank', 'is_goodprod', 'is_monday', 'is_weekend', 'is_after_16',
       'is_earned_income1', 'is_earned_income2', 'is_job_regular',
       'is_job_etc', 'is_house_own', 'is_house_lent', 'is_purpose_living',
       'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn',
       'macro_pred']]


result = pd.concat([vld_0, micro_df])   
# macro에서 0인것들의 모음 vld_0과 1인것들만 따로 모아서 예측을 실행해 만든 micro_df를 위아래 concat으로 병합
# 이 때, micro_pred 열이름을 그냥 macro_pred로 변환시킴
# 그러므로 macro_pred에 있는 값이 최종 결과로서의 역할을 함
result = pd.merge(vld_total, result, on = ['application_id', 'product_id'] )
# 최종적으로 합친 result에는 행들이 뒤섞여 있으므로 merge를 통해서 app_id와 pro_id를 기준으로 병합
result.columns
result = result.drop(['credit_score_y',
       'application_n_y', 'loyalty_y', 'gender_y', 'log_yearly_income_y',
       'log_career_y', 'log_desired_amount_y', 'log_existing_loan_cnt_y',
       'log_existing_loan_amt_y', 'log_OpenApp_y', 'log_UseDSRCalc_y',
       'nl_loan_limit_y', 'nl_loan_rate_y', 'nl_age_y', 'is_goodbank_y',
       'is_goodprod_y', 'is_monday_y', 'is_weekend_y', 'is_after_16_y',
       'is_earned_income1_y', 'is_earned_income2_y', 'is_job_regular_y',
       'is_job_etc_y', 'is_house_own_y', 'is_house_lent_y',
       'is_purpose_living_y', 'personal_rehabilitation_yn_y',
       'personal_rehabilitation_complete_yn_y'], axis = 1)

# 중복되는 행 삭제
result



from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
print(accuracy_score(result['is_applied'], result['macro_pred']),
      f1_score(result['is_applied'], result['macro_pred']),
      recall_score(result['is_applied'], result['macro_pred']),
      precision_score(result['is_applied'], result['macro_pred']))

ValueError: Feature shape mismatch, expected: 29, got 28

In [61]:
from sqlite3 import adapt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
###################################################################################################################

## macro 데이터 불러오기
trn_macro = pd.read_csv('C:\\Users\\zzuge\\trn_macro.csv')
vld_macro = pd.read_csv('C:\\Users\\zzuge\\vld_macro.csv')

## micro 데이터 불러오기
trn_micro = pd.read_csv('C:\\Users\\zzuge\\trn_micro.csv')
vld_micro = pd.read_csv('C:\\Users\\zzuge\\vld_micro.csv')

trn_macro = trn_macro.drop(['product_id','application_id'], axis = 1)
trn_micro = trn_micro.drop(['product_id','application_id'],axis = 1)
vld_macro = vld_macro.drop(['product_id','application_id'], axis = 1)
vld_micro = vld_micro.drop(['product_id','application_id'], axis = 1)


trn_macro_x = trn_macro.drop('is_applied' ,axis = 1)
trn_macro_y = trn_macro.loc[:,'is_applied']
trn_micro_x = trn_micro.drop('is_applied', axis = 1)
trn_micro_y = trn_micro.loc[:,'is_applied']


vld_total = pd.read_csv('C:\\Users\\zzuge\\vld_total.csv')


In [62]:
real = vld_total['is_applied']
vld_df = vld_total.drop(['is_applied', 'product_id'], axis = 1)
vld_df = vld_df.groupby('application_id').agg('max')
ad = vld_df.index
vld_df = vld_df.reset_index().drop('application_id', axis=1)

In [63]:
vld_df

,credit_score,application_n,loyalty,gender,log_yearly_income,log_career,log_desired_amount,log_existing_loan_cnt,log_existing_loan_amt,log_OpenApp,...,is_after_16,is_earned_income1,is_earned_income2,is_job_regular,is_job_etc,is_house_own,is_house_lent,is_purpose_living,personal_rehabilitation_yn,personal_rehabilitation_complete_yn
0,590,11,3,1,7.361728,2.630428,7.301030,0.301030,6.954243,0.000000,...,0,1,0,0,0,0,0,0,0,0
1,870,12,0,0,7.505150,3.690373,8.176091,0.000000,7.732394,1.113943,...,0,0,0,0,1,1,0,0,0,0
2,550,1,9,0,7.447158,3.346939,7.447158,0.778151,7.612784,1.176091,...,0,1,0,1,0,1,0,1,0,0
3,720,4,13,1,7.653213,3.370143,7.505150,0.301030,7.770852,2.000000,...,0,1,0,1,0,0,1,1,0,0
4,750,4,0,1,7.602060,2.437751,7.698970,0.000000,7.653213,0.301030,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106845,610,14,9,1,7.431364,2.260071,7.301030,0.477121,7.322219,1.204120,...,0,1,0,1,0,0,1,1,0,0
106846,650,2,6,1,7.397940,2.598791,6.477121,0.477121,7.477121,0.954243,...,1,0,0,0,1,0,0,0,0,0
106847,670,25,2,0,7.414973,3.328583,6.698970,0.698970,7.602060,1.041393,...,1,1,0,1,0,0,0,1,0,0
106848,790,46,3,1,7.908485,3.894980,8.000000,0.698970,7.973128,1.477121,...,0,1,0,1,0,0,0,1,0,0


In [64]:
###################################################################################################################
## 데이터 학습과정

## test데이터를 macro데이터같은 모양새를 띄도록 변경

real = vld_total['is_applied']
vld_df = vld_total.drop(['is_applied', 'product_id'], axis = 1)
vld_df = vld_df.groupby('application_id').agg('max')
ad = vld_df.index
vld_df = vld_df.reset_index().drop('application_id', axis=1)


## LGBM으로 is_applied 예측해서 하나라도 1이 있는 행들만 모음
## vld_df : application_id와 macro_pred가 합쳐진 데이터프레임
###################################################################################################
macro_lgb = XGBClassifier(random_state = 121, n_estimators = 10000)## 분석변경구간#################
#################################################################################################
macro_lgb.fit(trn_macro_x, trn_macro_y)
macro_pred = macro_lgb.predict(vld_df)
macro_pred = pd.DataFrame(macro_pred)
ad = pd.DataFrame(ad)

vld_df = pd.concat([vld_df, ad], axis = 1)
vld_df = pd.concat([vld_df, macro_pred], axis = 1)
vld_df.rename(columns = {0:'macro_pred'}, inplace = True)
vld_df = pd.merge(vld_total, vld_df, left_on = 'application_id', right_on = 'application_id', how = 'inner')

In [65]:
# 나중에 0인자리는 두고 1인 자리에 micro의 예측값을 채워야함
vld_df = vld_df.drop(['credit_score_y',
       'application_n_y', 'loyalty_y', 'gender_y', 'log_yearly_income_y',
       'log_career_y', 'log_desired_amount_y', 'log_existing_loan_cnt_y',
       'log_existing_loan_amt_y', 'log_OpenApp_y', 'log_UseDSRCalc_y',
       'nl_loan_limit_y', 'nl_loan_rate_y', 'nl_age_y', 'is_goodbank_y',
       'is_goodprod_y', 'is_monday_y', 'is_weekend_y', 'is_after_16_y',
       'is_earned_income1_y', 'is_earned_income2_y', 'is_job_regular_y',
       'is_job_etc_y', 'is_house_own_y', 'is_house_lent_y',
       'is_purpose_living_y', 'personal_rehabilitation_yn_y',
       'personal_rehabilitation_complete_yn_y'], axis = 1)

# micro 데이터를 학습을 위한 모양으로 변경
vld_micro_1 = vld_df[vld_df['macro_pred'] == 1]  ## macro 데이터에서 1인것들만 골라서 vld_micro라고 하자
ad = vld_micro_1['application_id']
prd = vld_micro_1['product_id']
## application_id와 product_id와 macro_pred를 빼고 분석시행
vld_micro_1 = vld_micro_1.drop(['application_id', 'product_id', 'macro_pred'], axis = 1)
vld_micro_1 = vld_micro_1.drop('is_applied', axis =1 )
vld_micro_1.columns = ['credit_score', 'application_n', 'loyalty', 'gender',
       'log_yearly_income', 'log_career', 'log_desired_amount',
       'log_existing_loan_cnt', 'log_existing_loan_amt', 'log_OpenApp',
       'log_UseDSRCalc', 'nl_loan_limit', 'nl_loan_rate', 'nl_age',
       'is_goodbank', 'is_goodprod', 'is_monday', 'is_weekend', 'is_after_16',
       'is_earned_income1', 'is_earned_income2', 'is_job_regular',
       'is_job_etc', 'is_house_own', 'is_house_lent', 'is_purpose_living',
       'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn']

In [66]:
## LGBM으로 macro에서 걸러낸 micro데이터의 is_applied를 예측
#################################################################################################
lgb_micro = XGBClassifier(random_state = 121, n_estimators = 10000)##분석변경구간##############
################################################################################################
lgb_micro.fit(trn_micro_x,  trn_micro_y)
micro_pred = lgb_micro.predict(vld_micro_1)

In [67]:
micro_pred

array([0, 0, 0, ..., 0, 0, 0])

In [70]:
# micro_pred와 vld_micro를 합쳐서 micro_df라고 하자
micro_4_pred = pd.DataFrame(micro_pred)
micro_4_pred.rename(columns = {0: 'micro_pred'}, inplace  = True)
vld_micro.reset_index(drop = True, inplace = True)
micro_4_pred.reset_index(drop = True, inplace = True)
vld_micro_1.index = micro_4_pred.index  # concat과정에서 index가 달라서 nan이 발생해서 인덱스를 동일하게 함
micro_df = pd.concat([vld_micro_1, micro_4_pred], axis =1)

In [71]:
## micro  결과까지 vld_micro데이터프레임에 붙여놓은 상태
# 이제 다시 vld_total 과 같은 형태를 만들어야함

ad.index = micro_df.index
prd.index = micro_df.index  # 떼어놓았던 application_id와 product_id를 다시 micro_df에 병합
micro_df = pd.concat([micro_df, ad], axis = 1 )
micro_df = pd.concat([micro_df, prd], axis = 1 )
micro_df


## vld_df에 1이 있는 곳에 micro_df의 micro_pred 의 값을 채우고 0인곳을 그대로 둔다
vld_0 = vld_df[vld_df['macro_pred']!=1] # macro_pred의 값이 0이 아닌 데이터들을 모은 것
vld_0.drop('is_applied', axis =1 , inplace = True)
vld_0.columns = ['application_id', 'product_id', 'credit_score', 'application_n', 'loyalty', 'gender',
       'log_yearly_income', 'log_career', 'log_desired_amount',
       'log_existing_loan_cnt', 'log_existing_loan_amt', 'log_OpenApp',
       'log_UseDSRCalc', 'nl_loan_limit', 'nl_loan_rate', 'nl_age',
       'is_goodbank', 'is_goodprod', 'is_monday', 'is_weekend', 'is_after_16',
       'is_earned_income1', 'is_earned_income2', 'is_job_regular',
       'is_job_etc', 'is_house_own', 'is_house_lent', 'is_purpose_living',
       'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn', 'macro_pred']

vld_0.reset_index(drop = True, inplace = True)
micro_df.columns
micro_df.rename(columns = {'micro_pred': 'macro_pred'}, inplace = True)
micro_df = micro_df[['application_id', 'product_id','credit_score', 'application_n', 'loyalty', 'gender',
       'log_yearly_income', 'log_career', 'log_desired_amount',
       'log_existing_loan_cnt', 'log_existing_loan_amt', 'log_OpenApp',
       'log_UseDSRCalc', 'nl_loan_limit', 'nl_loan_rate', 'nl_age',
       'is_goodbank', 'is_goodprod', 'is_monday', 'is_weekend', 'is_after_16',
       'is_earned_income1', 'is_earned_income2', 'is_job_regular',
       'is_job_etc', 'is_house_own', 'is_house_lent', 'is_purpose_living',
       'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn',
       'macro_pred']]


result = pd.concat([vld_0, micro_df])   
# macro에서 0인것들의 모음 vld_0과 1인것들만 따로 모아서 예측을 실행해 만든 micro_df를 위아래 concat으로 병합
# 이 때, micro_pred 열이름을 그냥 macro_pred로 변환시킴
# 그러므로 macro_pred에 있는 값이 최종 결과로서의 역할을 함
result = pd.merge(vld_total, result, on = ['application_id', 'product_id'] )
# 최종적으로 합친 result에는 행들이 뒤섞여 있으므로 merge를 통해서 app_id와 pro_id를 기준으로 병합
result.columns
result = result.drop(['credit_score_y',
       'application_n_y', 'loyalty_y', 'gender_y', 'log_yearly_income_y',
       'log_career_y', 'log_desired_amount_y', 'log_existing_loan_cnt_y',
       'log_existing_loan_amt_y', 'log_OpenApp_y', 'log_UseDSRCalc_y',
       'nl_loan_limit_y', 'nl_loan_rate_y', 'nl_age_y', 'is_goodbank_y',
       'is_goodprod_y', 'is_monday_y', 'is_weekend_y', 'is_after_16_y',
       'is_earned_income1_y', 'is_earned_income2_y', 'is_job_regular_y',
       'is_job_etc_y', 'is_house_own_y', 'is_house_lent_y',
       'is_purpose_living_y', 'personal_rehabilitation_yn_y',
       'personal_rehabilitation_complete_yn_y'], axis = 1)

# 중복되는 행 삭제
result



from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
print(accuracy_score(result['is_applied'], result['macro_pred']),
      f1_score(result['is_applied'], result['macro_pred']),
      recall_score(result['is_applied'], result['macro_pred']),
      precision_score(result['is_applied'], result['macro_pred']))

C:\Users\zzuge\AppData\Local\Temp\ipykernel_2432\341312865.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vld_0.drop('is_applied', axis =1 , inplace = True)


0.9397459154648291 0.40603734824600196 0.34935559101509756 0.4846740685239425
